In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
service = Service(executable_path=ChromeDriverManager().install())
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

In [3]:
url = 'https://tickets.interpark.com/contents/genre/musical'
html = urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

In [4]:
musical = soup.find_all('div', {'class':'Card_cardContent__b557a'})
len(musical)

15

In [5]:
musical[0]

<div class="Card_cardContent__b557a"><div class="DiscountTicket_saleBadge__b15ac"><span>프리뷰 할인</span></div><ul class="DiscountTicket_wrapContents__342ce"><li class="DiscountTicket_ticketOpenTitle__e8774">붉은 정원</li><li class="DiscountTicket_concertHallName__e3a96">링크아트센터드림 드림2관</li><li class="DiscountTicket_playPeriod__6ab6f">2025.4.22 ~ 7.20</li><li class="DiscountTicket_wrapRow__805aa"><span class="DiscountTicket_discountRate__c49b7">50%</span><span class="DiscountTicket_discountPrice__1dc95">33,000원</span></li></ul></div>

In [6]:
discount = musical[0].find_all('span')
discount

[<span>프리뷰 할인</span>,
 <span class="DiscountTicket_discountRate__c49b7">50%</span>,
 <span class="DiscountTicket_discountPrice__1dc95">33,000원</span>]

In [7]:
info = musical[0].find_all('li')
info

[<li class="DiscountTicket_ticketOpenTitle__e8774">붉은 정원</li>,
 <li class="DiscountTicket_concertHallName__e3a96">링크아트센터드림 드림2관</li>,
 <li class="DiscountTicket_playPeriod__6ab6f">2025.4.22 ~ 7.20</li>,
 <li class="DiscountTicket_wrapRow__805aa"><span class="DiscountTicket_discountRate__c49b7">50%</span><span class="DiscountTicket_discountPrice__1dc95">33,000원</span></li>]

In [8]:
info_list = [i.text for i in info][:-1]
info_list

['붉은 정원', '링크아트센터드림 드림2관', '2025.4.22 ~ 7.20']

In [9]:
dc_list = [dc.text for dc in discount]
dc_list

['프리뷰 할인', '50%', '33,000원']

In [11]:
# 딕셔너리로 만드는 함수
def make_dic(tag_list, info_list):
    musical_dic = {}
    for t, i in zip(tag_list, info_list):
        musical_dic[t] = i
    return musical_dic

In [12]:
tag_list = ['제목', '장소', '기간', '할인유형', '할인률', '할인가']
info_list = info_list + dc_list
make_dic(tag_list, info_list)

{'제목': '붉은 정원',
 '장소': '링크아트센터드림 드림2관',
 '기간': '2025.4.22 ~ 7.20',
 '할인유형': '프리뷰 할인',
 '할인률': '50%',
 '할인가': '33,000원'}

### 인터파크 할인 뮤지컬 정적 크롤링

#### 페이지 읽어오기

In [13]:
url = 'https://tickets.interpark.com/contents/genre/musical'
html = urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

#### 할인 뮤지컬 정보 수집

In [14]:
def make_info_list(mus):
    info_list = [i.text for i in mus.find_all('li')][:-1]
    dc_list = [dc.text for dc in mus.find_all('span')]
    info_list = info_list + dc_list
    return info_list

In [15]:
# 딕셔너리로 만드는 함수
def make_dic(tag_list, info_list):
    musical_dic = {}
    for t, i in zip(tag_list, info_list):
        musical_dic[t] = i
    return musical_dic

In [16]:
musical = soup.find_all('div', {'class':'Card_cardContent__b557a'})
tag_list = ['제목', '장소', '기간', '할인유형', '할인률', '할인가']
musical_list = []
for mus in musical:
    info_list = make_info_list(mus)
    musical_list.append(make_dic(tag_list, info_list))
pd.DataFrame(musical_list)

,제목,장소,기간,할인유형,할인률,할인가
0,붉은 정원,링크아트센터드림 드림2관,2025.4.22 ~ 7.20,프리뷰 할인,50%,"33,000원"
1,긴긴밤,인터파크 서경스퀘어 스콘2관,2025.3.12 ~ 5.25,마티네 할인,30%,"46,200원"
2,하트셉수트,대학로 자유극장,2025.3.11 ~ 6.1,청소년 할인,30%,"46,200원"
3,너의 결혼식,인터파크 유니플렉스 1관,2025.3.20 ~ 6.8,3차 조기예매 할인,30%,"46,200원"
4,라이카,두산아트센터 연강홀,2025.3.14 ~ 5.18,라이브러리컴퍼니 할인,20%,"56,000원"
5,종의 기원 콘서트,성신여대 운정그린캠퍼스 대강당,2025.4.20,뷰티풀웨이 할인,10%,"39,600원"
6,컨택트,극장 온(구 CJ아지트),2025.3.28 ~ 6.29,프리뷰 할인,50%,"33,000원"
7,2025 적벽,국립정동극장,2025.3.13 ~ 4.20,꽃청춘할인,40%,"42,000원"
8,니진스키,예스24아트원 1관,2025.3.25 ~ 6.15,2차 조기예매 할인,25%,"52,500원"
9,헝키쇼 (HUNKY SHOW),스타스테이지,2025.4.25 ~ 8.31,프리뷰 할인,55%,"39,600원"


In [18]:
df = pd.DataFrame(musical_list)
df.to_csv('crawl_data/뮤지컬할인.csv', index=False)

### 리뷰 페이지 동적 크롤링

In [19]:
# 리뷰 페이지 불러오기
driver = webdriver.Chrome(service=service)
url = 'https://ticket.interpark.com/Community/Play/Talk/main.asp'
driver.get(url)
driver.implicitly_wait(3)
time.sleep(1)
xpath = '/html/body/table/tbody/tr[5]/td/table/tbody/tr/td[1]/table[1]/tbody/tr/td/table/tbody/tr/td[3]/a'
driver.find_element(By.XPATH, xpath).click()
time.sleep(1)
xpath = '/html/body/table/tbody/tr[4]/td/table/tbody/tr/td[1]/table/tbody/tr[3]/td/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td[2]/table/tbody/tr[1]/td[2]/a'
driver.find_element(By.XPATH, xpath).click()

In [20]:
music_elem = driver.find_element(By.ID, 'selGoodsName')
music_elem.send_keys('뮤지컬 〈도리안 그레이〉')

In [21]:
score = driver.find_element(By.CLASS_NAME, "textsmall")
score.text

'(5.2점/10점)'

In [14]:
u_element = driver.find_element(By.TAG_NAME, "u")
u_element.text

'2025.03.30~2025.06.08 홍익대 대학로 아트센터 대극장'

#### 뮤지컬 별점 데이터 전체 수집

In [23]:
# 리뷰 페이지 불러오기
driver = webdriver.Chrome(service=service)
url = 'https://ticket.interpark.com/Community/Play/Talk/main.asp'
driver.get(url)
driver.implicitly_wait(3)
time.sleep(1)
xpath = '/html/body/table/tbody/tr[5]/td/table/tbody/tr/td[1]/table[1]/tbody/tr/td/table/tbody/tr/td[3]/a'
driver.find_element(By.XPATH, xpath).click()
time.sleep(1)
xpath = '/html/body/table/tbody/tr[4]/td/table/tbody/tr/td[1]/table/tbody/tr[3]/td/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td[2]/table/tbody/tr[1]/td[2]/a'
driver.find_element(By.XPATH, xpath).click()

In [24]:
# 뮤지컬 이름 리스트 생성
xpath = '//*[@id="selGoodsName"]'
musicList = driver.find_element(By.XPATH, xpath)
musicOpt = musicList.find_elements(By.TAG_NAME, 'option')
musicNames = [opt.text for opt in musicOpt][1:]

In [25]:
# print(musicNames)

In [26]:
# 뮤지컬 제목, 장소 기간, 평점을 튜플로 출력하는 함수
def search_score(title):
    music_elem = driver.find_element(By.ID, 'selGoodsName')
    music_elem.send_keys(title)
    score = driver.find_element(By.CLASS_NAME, "textsmall")
    u_element = driver.find_element(By.TAG_NAME, "u")
    return title, u_element.text, score.text

In [27]:
title_list = ['뮤지컬 〈도리안 그레이〉', '뮤지컬 〈긴긴밤〉', '뮤지컬 〈알라딘〉 한국 초연 (ALADDIN The Musical)']

In [28]:
score_list = []
for title in title_list:
    score_dic = {}
    score = search_score(title)
    score_dic['제목'] = score[0]
    score_dic['장소/기간'] = score[1]
    score_dic['평점'] = score[2]
    time.sleep(7)
    score_list.append(score_dic)
score_list

[{'제목': '뮤지컬 〈도리안 그레이〉',
  '장소/기간': '2025.03.30~2025.06.08 홍익대 대학로 아트센터 대극장',
  '평점': '(5.2점/10점)'},
 {'제목': '뮤지컬 〈긴긴밤〉',
  '장소/기간': '2025.03.12~2025.05.25 인터파크 서경스퀘어 스콘2관',
  '평점': '(9.8점/10점)'},
 {'제목': '뮤지컬 〈알라딘〉 한국 초연 (ALADDIN The Musical)',
  '장소/기간': '2024.11.22~2025.06.22 샤롯데씨어터',
  '평점': '(9.3점/10점)'}]

In [16]:
pd.DataFrame(score_list)

,제목,장소/기간,평점
0,뮤지컬 〈도리안 그레이〉,2025.03.30~2025.06.08 홍익대 대학로 아트센터 대극장,(5.4점/10점)
1,뮤지컬 〈긴긴밤〉,2025.03.12~2025.05.25 인터파크 서경스퀘어 스콘2관,(9.8점/10점)
2,뮤지컬 〈알라딘〉 한국 초연 (ALADDIN The Musical),2024.11.22~2025.06.22 샤롯데씨어터,(9.3점/10점)


In [29]:
people = driver.find_element(By.XPATH, "//*[contains(@style, 'color:#495692')]")
people.text

'총2259명'

In [30]:
def search_sc(title):
    music_elem = driver.find_element(By.ID, 'selGoodsName')
    music_elem.send_keys(title)
    score = driver.find_element(By.CLASS_NAME, "textsmall")
    people = driver.find_element(By.XPATH, "//*[contains(@style, 'color:#495692')]")
    u_element = driver.find_element(By.TAG_NAME, "u")
    return title, u_element.text, people.text, score.text
search_sc('뮤지컬 〈알라딘〉 한국 초연 (ALADDIN The Musical)')

('뮤지컬 〈알라딘〉 한국 초연 (ALADDIN The Musical)',
 '2024.11.22~2025.06.22 샤롯데씨어터',
 '총2259명',
 '(9.3점/10점)')

In [31]:
# 평점 리스트 만드는 함수
def make_list(title_list):
    score_list = []
    for title in title_list:
        score_dic = {}
        score = search_sc(title)
        score_dic['제목'] = score[0]
        score_dic['장소/기간'] = score[1]
        score_dic['참여자 수'] = score[2]
        score_dic['평점'] = score[3]
        time.sleep(7)
        score_list.append(score_dic)
    return score_list

In [42]:
title_list = ['뮤지컬 〈도리안 그레이〉', '뮤지컬 〈긴긴밤〉', '뮤지컬 〈알라딘〉 한국 초연 (ALADDIN The Musical)']
df = pd.DataFrame(make_list(title_list))

In [43]:
df.to_csv('crawl_data/뮤지컬평점.csv', index=False)